In [1]:
from loader import load_reviews_to_list
from text_search import *
from spatial_search import *
from spatio_textual_search import *

In [2]:
path = 'Data\\Restaurants_London_England.tsv'
restaurants_list, borders = load_reviews_to_list(path)

tags_list, bags_of_restaurants = generate_inverted_index(restaurants_list)

x_list, y_list = create_grid(borders,50)
grid = add_restaurants_to_grid(x_list, y_list, restaurants_list)

In [5]:
r_query = [51, 51.50, -0.5, 0]
t_query = ['british', 'bar']
#kwspaif =  kwSpaSearchGrid(r_query, t_query, tags_list, bags_of_restaurants, x_list, y_list, grid, restaurants_list)
kwspaif = kwSpaSearchRaw(r_query, t_query, restaurants_list)

In [6]:
print(len(kwspaif))
for r in kwspaif:
    print(restaurants_list[r]['review_id'])
    print('location:', restaurants_list[r]['x'], ',', restaurants_list[r]['y'])
    print('tags:', restaurants_list[r]['tags'], '\n')

5
Restaurant_Review-g186338-d944622-Reviews-Yacht-London_England.html
location: 51.484703 , -0.00394
tags: ['english', 'british', 'bar', 'grill'] 

Restaurant_Review-g186338-d734073-Reviews-Castle-London_England.html
location: 51.47279 , -0.173312
tags: ['british', 'bar', 'grill'] 

Restaurant_Review-g186338-d1017689-Reviews-Society_Bar_Restaurant-London_England.html
location: 51.496468 , -0.206702
tags: ['contemporary', 'international', 'british', 'bar', 'reservations', 'private dining'] 

Restaurant_Review-g186338-d806450-Reviews-THE_TERRACE_KITCHEN_BAR-London_England.html
location: 51.412285 , -0.12382
tags: ['caribbean', 'vegetarian', 'british', 'bar', 'families with children', 'romance', 'outdoor seating', 'breakfast/brunch'] 

Restaurant_Review-g186338-d817313-Reviews-Barstory-London_England.html
location: 51.46933 , -0.070317
tags: ['british', 'bar', 'bistro', 'romance', 'outdoor seating'] 



In [9]:
borders

{'min_x': 50.546856, 'max_x': 55.95297, 'min_y': -4.243601, 'max_y': 1.387973}

In [12]:
for i,t in enumerate(tags_list):
    if len(bags_of_restaurants[i]) > 100:
        print(t, len(bags_of_restaurants[i]))

american 315
asian 205
bakery 105
bar 344
bar scene 395
breakfast/brunch 1380
british 1346
cafÃ© 165
chinese 499
coffee shop 355
contemporary 164
delivery 466
dessert 261
dining on a budget 146
dinner 972
doing business 353
english 714
entertaining clients 337
european 602
families with children 380
french 499
grill 276
indian 936
international 298
italian 1386
japanese 339
large groups 333
late night 2062
lebanese 149
local cuisine 160
lunch 826
mediterranean 327
mexican 127
outdoor seating 256
pizza 150
pub 147
reservations 3012
romance 404
seafood 112
spanish 166
special occasions 353
takeout 338
thai 368
turkish 240
vegetarian 185
